In [3]:
# import torch
# path = '/home/jgryu/workspace/weight_compression/hf_model_comp/comp_qtip/ckpt/meta-llama--Meta-Llama-3-8B/ql_ldlq128_rnorm_codes_test/lmbda50/0_up.pt'

# s = torch.load(path, weights_only=False, map_location='cpu')

import torch
# path = '/home/jgryu/workspace/weight_compression/complexity_test/8b_qtip_ft1_4bit.pth'
path = '/home/jgryu/workspace/weight_compression/complexity_test/8b-w4-g128-awq-v2.pt'
s = torch.load(path)

/tmp/ipykernel_665459/2865850411.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  s = torch.load(path)


In [4]:
for k, v in s.items():
    print(k, v.shape, v.dtype)

model.embed_tokens.weight torch.Size([128256, 4096]) torch.float16
model.layers.0.self_attn.q_proj.qweight torch.Size([1024, 4096]) torch.int16
model.layers.0.self_attn.q_proj.scales torch.Size([32, 4096]) torch.float16
model.layers.0.self_attn.q_proj.scaled_zeros torch.Size([32, 4096]) torch.float16
model.layers.0.self_attn.k_proj.qweight torch.Size([256, 4096]) torch.int16
model.layers.0.self_attn.k_proj.scales torch.Size([32, 1024]) torch.float16
model.layers.0.self_attn.k_proj.scaled_zeros torch.Size([32, 1024]) torch.float16
model.layers.0.self_attn.v_proj.qweight torch.Size([256, 4096]) torch.int16
model.layers.0.self_attn.v_proj.scales torch.Size([32, 1024]) torch.float16
model.layers.0.self_attn.v_proj.scaled_zeros torch.Size([32, 1024]) torch.float16
model.layers.0.self_attn.o_proj.qweight torch.Size([1024, 4096]) torch.int16
model.layers.0.self_attn.o_proj.scales torch.Size([32, 4096]) torch.float16
model.layers.0.self_attn.o_proj.scaled_zeros torch.Size([32, 4096]) torch.flo

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"

: 

### NWC state_dict 만들기

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

from transformers import LlamaForCausalLM

path = '/home/jgryu/workspace/weight_compression/hf_model_comp/comp_qtip/ckpt/meta-llama--Meta-Llama-3-8B/ql_ldlq128_rnorm_codes_test/lmbda300'
codes_dict = {}
metadata_dict = {}

n_parm = 0
n_bit = 0

for i in range(32):
    for m in ['q', 'k', 'v', 'o', 'up', 'down', 'gate']:
        pt = f'{path}/{i}_{m}.pt'
        s = torch.load(pt, weights_only=False, map_location='cpu')

        codes = s['codes']
        tensor_code_list = []
        for c in codes:
            # print(len(c['strings'][0][0]))
            byte_stream = c['strings'][0][0]
            tensor_view = torch.frombuffer(byte_stream, dtype=torch.uint8)
            tensor_code_list.append(tensor_view)
            # print(tensor_view.shape)
        
        padded_code_tensor = pad_sequence(tensor_code_list, batch_first=True, padding_value=0)
        n_bit += padded_code_tensor.numel() * 8
        
        
        key = f'{i}_{m}'
        
        codes_dict[key] = padded_code_tensor
        codes_dict[key + '_row_std'] = s['metadata']['row_std']
        codes_dict[key + '_qlevel'] = s['metadata']['qlevel'].to(dtype = torch.uint8)
        
        metadata_dict[key] = {
            'row_std': s['metadata']['row_std'],
            'qlevel': s['metadata']['qlevel'].to(dtype = torch.uint8),
            'shape': (128, int(c['shape'][0] / 128 * 16))
        }

path = '/home/jgryu/workspace/weight_compression/Wparam_dataset/hf_model/meta-llama--Meta-Llama-3-8B'
model = LlamaForCausalLM.from_pretrained(
    path,
    torch_dtype=torch.float16,
    device_map='cpu')
# torch.save(model.state_dict(), './8b_sd_original.pth')

original_sd = model.state_dict()
keys_to_keep = []
for k in original_sd.keys():
    if (
        "layernorm" in k.lower()
        or 'embed' in k.lower()
        or k == "model.norm.weight"
        or k == "lm_head.weight"
    ):
        keys_to_keep.append(k)
    else:
        n_parm += original_sd[k].numel()
for k in list(original_sd.keys()):
    if k not in keys_to_keep:
        del original_sd[k]

codes_dict.update(original_sd)

# 4. 두 딕셔너리를 별도의 파일로 저장
# torch.save(codes_dict, './8b_compressed_ld300_codes.pth')
# torch.save(metadata_dict, './8b_compressed_ld300_metadata.pth')

: 

In [5]:
n_bit/n_parm

4.056882564838116

### 시간측정 함수 정의

In [2]:
import time
import torch
import os
import argparse
import numpy as np # 평균 계산을 위해 추가

def measure_load_time(model_path, device):
    """
    .pth 파일을 로드하고 GPU로 전송하는 시간을 측정하여 ms 단위로 반환합니다.
    이 함수는 캐시 상태(hot/cold)를 알지 못합니다.
    """

    # --- 0. 입력 검증 ---
    if not os.path.exists(model_path):
        print(f"  오류: {model_path} 파일을 찾을 수 없습니다.")
        return None, None # 실패 시 None 반환

    if device.type == 'cpu':
        print("  오류: CUDA 디바이스가 필요합니다.")
        return None, None # 실패 시 None 반환
        
    if not torch.cuda.is_available():
        print("  오류: CUDA를 사용할 수 없습니다.")
        return None, None

    # --- 1. Disk -> CPU RAM 시간 측정 (CPU 작업) ---
    torch.cuda.synchronize(device) # 이전 GPU 작업 완료 보장
    start_cpu_time = time.perf_counter()
    
    # torch.load는 I/O와 역직렬화(CPU 작업)를 수행합니다.
    state_dict_cpu = torch.load(model_path, map_location='cpu', weights_only=False)
        
    end_cpu_time = time.perf_counter()
    torch.cuda.synchronize(device)
    
    load_time_ms = (end_cpu_time - start_cpu_time) * 1000
    # print(f"  [1] Disk/Cache -> CPU RAM 시간: {load_time_ms:.2f} ms") # <-- 이제 반환값으로 처리

    # --- 2. CPU RAM -> GPU VRAM 시간 측정 (GPU 작업) ---
    start_gpu_event = torch.cuda.Event(enable_timing=True)
    end_gpu_event = torch.cuda.Event(enable_timing=True)
    
    start_gpu_event.record()
    
    # state_dict의 모든 텐서를 GPU로 이동시킵니다.
    state_dict_gpu = {k: v.to(device) for k, v in state_dict_cpu.items()}
    
    end_gpu_event.record()
    
    # GPU 작업이 끝날 때까지 CPU가 기다리도록 강제
    torch.cuda.synchronize(device)
    
    transfer_time_ms = start_gpu_event.elapsed_time(end_gpu_event)
    # print(f"  [2] CPU RAM -> GPU VRAM 시간: {transfer_time_ms:.2f} ms") # <-- 이제 반환값으로 처리
    
    # 측정 후 메모리 정리
    del state_dict_cpu, state_dict_gpu
    torch.cuda.empty_cache()
    
    return load_time_ms, transfer_time_ms

# --- 메인 실행 로직 ---


In [3]:
paths = [
    '8b_sd_original.pth',
    '8b_compressed_ld300_codes.pth',
    '8b_qtip_noft_4bit.pth',
    '8b-w4-g128-awq-v2.pt',
]
N_RUNS = 5 # <--- 여기에서 반복 횟수를 조절하세요
DEVICE = torch.device('cuda:2')

for p in paths:
    print(f"\n[측정 대상: {p}]")
    
    load_times_ms = []
    transfer_times_ms = []
    
    for i in range(N_RUNS):
        torch.cuda.empty_cache()
        
        load_ms, transfer_ms = measure_load_time(p, DEVICE)
        
        if load_ms is not None and transfer_ms is not None:
            # print(f"    Run {i+1}/{N_RUNS}: Load={load_ms:.2f}ms, Transfer={transfer_ms:.2f}ms, Total={load_ms+transfer_ms:.2f}ms")
            load_times_ms.append(load_ms)
            transfer_times_ms.append(transfer_ms)
        else:
            print(f"    Run {i+1}/{N_RUNS}: 측정 실패")

    if load_times_ms: # 성공한 측정이 하나라도 있는 경우
        avg_load = np.mean(load_times_ms)
        avg_transfer = np.mean(transfer_times_ms)
        avg_total = avg_load + avg_transfer
        
        print(f"  --- {p} 최종 결과 ({len(load_times_ms)}회 평균) ---")
        print(f"  [1] Disk/Cache -> CPU RAM 시간 (평균): {avg_load/1000:.3f} s")
        print(f"  [2] CPU RAM -> GPU VRAM 시간 (평균): {avg_transfer/1000:.3f} s")
        print(f"  -------------------------------------")
        print(f"  총합 (평균): {avg_total/1000:.3f} s")
    else:
        print(f"  --- {p} 모든 측정 실행 실패 ---")

print("\n--- 모든 측정 완료 ---")


[측정 대상: 8b_sd_original.pth]
  --- 8b_sd_original.pth 최종 결과 (5회 평균) ---
  [1] Disk/Cache -> CPU RAM 시간 (평균): 5.015 s
  [2] CPU RAM -> GPU VRAM 시간 (평균): 1.564 s
  -------------------------------------
  총합 (평균): 6.579 s

[측정 대상: 8b_compressed_ld300_codes.pth]
  --- 8b_compressed_ld300_codes.pth 최종 결과 (5회 평균) ---
  [1] Disk/Cache -> CPU RAM 시간 (평균): 1.379 s
  [2] CPU RAM -> GPU VRAM 시간 (평균): 0.813 s
  -------------------------------------
  총합 (평균): 2.192 s

[측정 대상: 8b_qtip_noft_4bit.pth]
  --- 8b_qtip_noft_4bit.pth 최종 결과 (5회 평균) ---
  [1] Disk/Cache -> CPU RAM 시간 (평균): 1.592 s
  [2] CPU RAM -> GPU VRAM 시간 (평균): 0.886 s
  -------------------------------------
  총합 (평균): 2.478 s

[측정 대상: 8b-w4-g128-awq-v2.pt]
  --- 8b-w4-g128-awq-v2.pt 최종 결과 (5회 평균) ---
  [1] Disk/Cache -> CPU RAM 시간 (평균): 1.145 s
  [2] CPU RAM -> GPU VRAM 시간 (평균): 1.195 s
  -------------------------------------
  총합 (평균): 2.341 s

--- 모든 측정 완료 ---


In [10]:
import torch
# path = '/home/jgryu/workspace/weight_compression/complexity_test/8b_qtip_noft_4bit.pth'
path = '/home/jgryu/workspace/weight_compression/complexity_test/8b-w4-g128-awq-v2.pt'
q = torch.load(path)

for k,v in q.items():
    print(k, v.shape, v.dtype)

/tmp/ipykernel_1290775/135903826.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  q = torch.load(path)


model.embed_tokens.weight torch.Size([128256, 4096]) torch.float16
model.layers.0.self_attn.q_proj.qweight torch.Size([1024, 4096]) torch.int16
model.layers.0.self_attn.q_proj.scales torch.Size([32, 4096]) torch.float16
model.layers.0.self_attn.q_proj.scaled_zeros torch.Size([32, 4096]) torch.float16
model.layers.0.self_attn.k_proj.qweight torch.Size([256, 4096]) torch.int16
model.layers.0.self_attn.k_proj.scales torch.Size([32, 1024]) torch.float16
model.layers.0.self_attn.k_proj.scaled_zeros torch.Size([32, 1024]) torch.float16
model.layers.0.self_attn.v_proj.qweight torch.Size([256, 4096]) torch.int16
model.layers.0.self_attn.v_proj.scales torch.Size([32, 1024]) torch.float16
model.layers.0.self_attn.v_proj.scaled_zeros torch.Size([32, 1024]) torch.float16
model.layers.0.self_attn.o_proj.qweight torch.Size([1024, 4096]) torch.int16
model.layers.0.self_attn.o_proj.scales torch.Size([32, 4096]) torch.float16
model.layers.0.self_attn.o_proj.scaled_zeros torch.Size([32, 4096]) torch.flo